In [ ]:
# %config IPCompleter.use_jedi = False

# 1 import packages, initialize and show map

In [ ]:
import ee
import geemap
import os
import random
import geemap.colormaps as cm
import numpy as np
import leafmap
from tqdm.notebook import tqdm

In [ ]:
ee.Initialize()

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
Map

In [ ]:
# aoi = Map.user_roi
# aoi

# 2 functions to expand array and split images using GDAL

In [ ]:
from osgeo import gdal
import os
import numpy as np

def pad_array_to_x64x64(array, dimension=64):
    """
    将数组的后两个维度扩充到 [dimension, dimension]，用 -999 填充。
    
    Args:
        array (np.ndarray): 输入的 NumPy 数组，维度为 [*, dimension, dimension]。

    Returns:
        np.ndarray: 扩充后的数组。
    """
    array= np.array(array)
    if array.shape[-2:] == (dimension, dimension):
        return array
    else:
        # 计算需要扩充的行数和列数
        rows_to_add = dimension - array.shape[-2]
        cols_to_add = dimension - array.shape[-1]

        # 使用 np.pad() 扩充数组，并用 -999 填充
        padded_array = np.pad(array, ((0, 0), (0, rows_to_add), (0, cols_to_add)), constant_values=-999)
        return padded_array


def crop_and_replace_nans_infs(input_path, output_folder, crop_size=64, overlap=0):
    '''
    按照重叠率裁剪多波段图像
    :param input_path:
    :param output_folder:
    :param crop_size:
    :param overlap:
    :return:
    '''
    # 打开输入的TIFF图像
    dataset = gdal.Open(input_path)

    # 获取图像的宽度和高度
    width = dataset.RasterXSize
    height = dataset.RasterYSize

    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)
    a = 0

    for i in range(0, width, int(crop_size * (1 - overlap))):
        for j in range(0, height, int(crop_size * (1 - overlap))):
            # 读取图像数据
            if width-i < 64 and height-j >64:
                image_data = dataset.ReadAsArray(i, j, width-i, crop_size)
            elif height-j < 64 and width-i > 64:
                image_data = dataset.ReadAsArray(i, j, crop_size, height-j)
            elif height-j < 64 and width-i < 64:
                image_data = dataset.ReadAsArray(i, j, width-i, height-j)
            else:
                image_data = dataset.ReadAsArray(i, j, crop_size, crop_size)

            # 检查NaN和Inf值并替换为0
            image_data = np.nan_to_num(image_data, copy=False, nan=-999, posinf=-999, neginf=-999)
            
            # 扩充后两个维度
            image_data = pad_array_to_x64x64(image_data)

            # 生成输出文件名，这里假设原始文件名是"input.tif"
            # output_filename = f"{output_folder}/{i}-{j}.tif"
            output_filename = f"{output_folder}/{int(i/64)}_{int(j/64)}.tif"
            # 创建一个新的TIFF文件
            driver = gdal.GetDriverByName("GTiff")
            output_dataset = driver.Create(output_filename, crop_size, crop_size, dataset.RasterCount, gdal.GDT_Float32)

            # 设置地理信息和投影信息
            output_dataset.SetGeoTransform((i, dataset.GetGeoTransform()[1], 0, j, 0, dataset.GetGeoTransform()[5]))
            output_dataset.SetProjection(dataset.GetProjection())

            # 将裁剪后的图像数据写入新文件
            for band_index in range(dataset.RasterCount):
                output_dataset.GetRasterBand(band_index + 1).WriteArray(image_data[band_index, :, :])

            # 关闭输出文件
            output_dataset = None
            a = a+1

    # 关闭输入文件
    dataset = None

# 3 ROI(region of interest)

In [ ]:
china_CL = ee.Image('users/dushuai/soil_parameters/CL')
Map.addLayer(china_CL, {}, 'china')
Map.center_object(china_CL, 4)

In [ ]:
scale = china_CL.projection().nominalScale()
mask = china_CL.select(['b1']).neq(-999)
geometry = china_CL.select(['b1']).geometry()

## 3.1 clip image collections to ROI and reproject

In [ ]:
def clip(image):
    image = image.reproject(**{'crs':'EPSG:4326','scale':scale}) \
                 .clipToBoundsAndScale(**{'geometry':geometry,
                                          'width':7560,'height':4320}) \
                 .mask(mask)
    return image

## 4 LAI

In [ ]:
# 设置时间范围
start_date = '2019-12-01'
end_date = '2021-01-29'

def add_day(image):
    day = ee.Date(image.get('system:time_start')).difference(ee.Date('2019-12-01'), 'day')
    image = image.addBands(ee.Image(day).rename('day')).set('day', day)
    return image

# 获取 ERA5-LAND 每日数据集
dataset_LAI = ee.ImageCollection('MODIS/061/MCD15A3H') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .select(['Lai']).map(add_day)

In [ ]:
LAI_len = dataset_LAI.size()
LAI_list = dataset_LAI.toList(LAI_len)
LAI_image = ee.Image(LAI_list.get(0)).addBands(mask)
LAI_image

In [ ]:
visualization = {
  'band': ['Lai'],
  'min': 0,
  'max': 100
}

In [ ]:
LAI_list

In [ ]:
Map.addLayer(LAI_image.mask(mask), visualization, 'LAI')

## 4.1 use linear interpolation method generate daily LAI data

In [ ]:
from datetime import datetime, timedelta

start_time = datetime(2020, 1, 1)
lai_images = ee.Image(0)

def Linear_inter(i):
    global lai_images
    global start_time
    
    image = ee.Image(LAI_list.get(i))
    lai_images = lai_images.addBands(image.select(['Lai']).rename(str(start_time.strftime("%Y-%m-%d"))).toDouble())
    inter_day = ee.Number(image.get('day')).add(1)
    
    previous = ee.Image(LAI_list.get(i))
    previous1 = ee.Image(LAI_list.get(i-1))
    previous2 = ee.Image(LAI_list.get(i-2))
    nexts = ee.Image(LAI_list.get(i+1))
    nexts1 = ee.Image(LAI_list.get(i+2))
    nexts2 = ee.Image(LAI_list.get(i+3))
    
    previous_img = previous.select(['Lai'])
    previous_day = previous.select(["day"])
    previous1_img = previous1.select(['Lai'])
    previous1_day = previous1.select(["day"])
    previous2_img = previous2.select(['Lai'])
    previous2_day = previous2.select(["day"])
    
    next_img = nexts.select(['Lai'])
    next_day = nexts.select(["day"])
    next1_img = nexts1.select(['Lai'])
    next1_day = nexts1.select(["day"])
    next2_img = nexts2.select(['Lai'])
    next2_day = nexts2.select(["day"])
    
    previous_all_img = previous_img.add(previous1_img).add(previous2_img).divide(ee.Image(3))
    previous_all_day = previous_day.add(previous1_day).add(previous2_day).divide(ee.Image(3))
    nexts_all_img = next_img.add(next1_img).add(next2_img).divide(ee.Image(3))
    nexts_all_day = next_day.add(next1_day).add(next2_day).divide(ee.Image(3))
    
    nexts_day = ee.Number(ee.Image(LAI_list.get(i+1)).get('day'))
    while inter_day.getInfo() != nexts_day.getInfo():
        start_time += timedelta(days=1)
        inter_img = nexts_all_img.subtract(previous_all_img)\
                            .divide(nexts_all_day.subtract(previous_all_day))\
                            .multiply(ee.Image(inter_day).subtract(previous_all_day))\
                            .add(previous_all_img)

        lai_images = lai_images.addBands(inter_img.rename(str(start_time.strftime("%Y-%m-%d"))).toDouble())
        inter_day = inter_day.add(1)
        
    start_time += timedelta(days=1)

In [ ]:
for num in range(8,100):
    Linear_inter(num)

In [ ]:
lai_images = lai_images.select(lai_images.bandNames().slice(1,)).reproject(**{'crs':'EPSG:4326','scale':scale}) \
                                              .clipToBoundsAndScale(**{'geometry':geometry,
                                                                      'width':7560,'height':4320})
lai_images

# 5 download LAI with fixed dimensions which we want using geemap 

In [ ]:
out = r'H:\soil_moistur_retrieval\images\lai.tif'
geemap.download_ee_image(lai_images, out, shape=[4320,7560])

# 6 split images to image chips with dimensions we want using leafmap if the image size is samll, like seviral hundreds MB.

In [ ]:
outdir = r'H:\soil_moistur_retrieval\tiles\lai'
leafmap.split_raster(out, out_dir=outdir, tile_size=(64,64), overlap=0)

# 7 split images to image chips with dimensions we want using GDAL if the image size is big, like more than several GB.

In [ ]:
# 按照重叠率裁剪多波段图像
from glob import glob
input_file = out
output_folder_path = r"H:\soil_moistur_retrieval\tiles\lai"
crop_and_replace_nans_infs(input_file, output_folder_path, overlap=0)

# 8 LAI time series change after interpolation and SG smoothing

In [ ]:
fc_point = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([112.37, 29.22]), {'name':'Cultivated Land'}),
    ee.Feature(ee.Geometry.Point([122.00, 49.80]), {'name':'Forest'}),# 东北
    ee.Feature(ee.Geometry.Point([112.88, 28.22]), {'name':'City'}),# 长沙
#     ee.Feature(ee.Geometry.Point([119.18, 29.84]), {'name':'Forest1'}),
])

lai = lai_images.sampleRegions(collection=fc_point)

In [ ]:
# list(data[0]['properties'].values())[:-1]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

data = lai.getInfo()['features']
# 示例数据
x_axis_data = list(range(1,367))
y_axis_data = list(data[0]['properties'].values())[:-1]
y_axis_data1 = list(data[1]['properties'].values())[:-1]
y_axis_data2 = list(data[2]['properties'].values())[:-1]
# y_axis_data3 = list(data[3]['properties'].values())[:-1]

# SG平滑 SG filter smooth
# window_length = 15
# polyorder = 2
# y_axis_data = savgol_filter(y_axis_data, window_length, polyorder)
# y_axis_data1 = savgol_filter(y_axis_data1, window_length, polyorder)
# y_axis_data2 = savgol_filter(y_axis_data2, window_length, polyorder)

# 绘制折线图
plt.plot(x_axis_data, y_axis_data, 'b-', alpha=0.5, linewidth=2, label=list(data[0]['properties'].values())[-1])
plt.plot(x_axis_data, y_axis_data1, 'r-', alpha=0.5, linewidth=2, label=list(data[1]['properties'].values())[-1])
plt.plot(x_axis_data, y_axis_data2, 'y-', alpha=0.5, linewidth=2, label=list(data[2]['properties'].values())[-1])
# plt.plot(x_axis_data, y_axis_data3, 'y-', alpha=0.5, linewidth=1, label=list(data[3]['properties'].values())[-1])
# 'b*--' 表示蓝色实线，数据点为实心原点标注

plt.legend()  # 显示图例
plt.xlabel('DOY')  # x轴标签
plt.ylabel('LAI')  # y轴标签

plt.show()
